In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#como el navegador a usar es chrome, hemos descargado ya el driver de la página, guardamos la ruta en una variable
#chrome_path = 'chromedriver_linux64/chromedriver'
#driver = webdriver.Chrome(executable_path = chrome_path)

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

In [3]:
eyecream_url = []

In [4]:
for i in range (1,14):
    #my_session = requests.session()
    url = f'https://www.stylevana.com/en_EU/skincare/eye-lip-care/eye-cream.html?p={i}' 
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.content, 'lxml')
    eyecream_list = soup.find_all('div', class_='product-item-info')
    for item in eyecream_list:
        for link in item.find_all('a', class_='product-image', href=True): 
            eyecream_url.append(link['href'])
print(len(eyecream_url))

156


with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

In [5]:
len(eyecream_url)

156

In [ ]:
class="product-badge-icon"

In [ ]:
for i in range (1,14):
    #my_session = requests.session()
    url = f'https://www.stylevana.com/en_EU/skincare/eye-lip-care/eye-cream.html?p={i}' 
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.content, 'lxml')
    try:
        name = soup.find('h2', class_="product-name").text.strip()
        bestseller = soup.find_all('img', class_='product-badge-icon')
        price = soup.find_all('span', class_="price")
    except:
        name = 'no info'
        bestseller = 'no'
        price = 'no info'

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [21]:
eyecream_url[382]

IndexError: list index out of range

In [6]:
eye_creams = []

In [7]:
for link in eyecream_url[0:154]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        name = soup.find('h1', class_='product-name-h1').text
        details = (soup.find('div', id='product-view-details').text.strip()) # details
        price = (soup.find('span', class_='price').text.strip())
        best_seller = soup.find(class_="product-badge-icon").get('title')
        ingredients = soup.find_all('div', id="product-view-ingredients") #ingredients
        for item in ingredients:
            ingredients = item.find('div', class_='panel-body').text.strip()
    except:
        details = 'no info'
        ingredients = 'no info'
        price = 'no info'
        best_seller = 'no'
        
    eyecream_info = {
    'name': name,
    'details': details,
    'ingredients': ingredients,
    'price': price,
    'best-seller': best_seller}
    
    eye_creams.append(eyecream_info)

In [8]:
print(len(eye_creams))

154


## 3. PASARLO A DATAFRAME

In [9]:
eyecreams_df = pd.DataFrame(eye_creams)
eyecreams_df

,name,details,ingredients,price,best-seller
0,iUNIK - Propolis Vitamin Eye Cream - 30ml,no info,no info,no info,no
1,PURITO - Centella Green Level Eye Cream - 30ml,Cruelty Free: Yes\n\n\nProduct Type: Eye Cream...,"Centella Asiatica Extract, Water, Hydrogenated...",7.69€,Best Sellers
2,innisfree - Perfect 9 Repair Eye Cream EX (201...,no info,no info,no info,no
3,MILKYDRESS - Wrinkle & Whitening Eye Cream,no info,no info,no info,no
4,Isntree - TW-Real Eye Cream - 30ml,Cruelty Free: Yes\n\n\nProduct Type: Eye Cream...,"Water, Butylene Glycol, Dipropylene Glycol, Gl...",8.39€,Cruelty Free
...,...,...,...,...,...
149,CosmetexRoland - Eye Treatment Cream N - Dark ...,no info,no info,no info,no
150,Dr. Different - Vitalift A Eye & Neck Night Tr...,no info,no info,no info,no
151,Etude House - Moistfull Collagen Eye Cream,no info,no info,no info,no
152,The Face Shop - The Therapy Secret Made Anti A...,no info,no info,no info,no


In [10]:
eyecreams = eyecreams_df.to_csv('../data_/eyecreams.csv')